In [1]:
data_dir = "../data/"
output_dir = "../dataset/"

In [2]:
# Import thư viện cần thiết
import os
import pandas as pd
from PIL import Image
import random
import shutil

# Đọc file CSV chứa nhãn
train_csv = pd.read_csv(os.path.join(data_dir, "Train.csv"))

# Tạo thư mục cho dữ liệu YOLO
os.makedirs(os.path.join(output_dir, "images/train"), exist_ok=True)
os.makedirs(os.path.join(output_dir, "labels/train"), exist_ok=True)
os.makedirs(os.path.join(output_dir, "images/val"), exist_ok=True)
os.makedirs(os.path.join(output_dir, "labels/val"), exist_ok=True)



In [3]:
# Số lượng lớp
classes = 43

# Tạo file data.yaml cho YOLO
with open(os.path.join(output_dir, "data.yaml"), "w") as f:
    f.write(f"train: {output_dir}images/train\n")
    f.write(f"val: {output_dir}images/val\n")
    f.write(f"nc: {classes}\n")
    f.write(f"names: {list(range(classes))}\n")

# Hàm tạo nhãn YOLO
def create_yolo_label(class_id, x1, y1, x2, y2, img_width, img_height, label_file):
    x_center = (x1 + x2) / 2 / img_width
    y_center = (y1 + y2) / 2 / img_height
    width = (x2 - x1) / img_width
    height = (y2 - y1) / img_height
    with open(label_file, "a") as f:
        f.write(f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")



In [4]:
# Xử lý ảnh
for index, row in train_csv.iterrows():
    try:
        img_path = os.path.join(data_dir, row['Path'])
        class_id = row['ClassId']
        width, height = row['Width'], row['Height']
        x1, y1, x2, y2 = row['Roi.X1'], row['Roi.Y1'], row['Roi.X2'], row['Roi.Y2']

        # Mở ảnh
        image = Image.open(img_path).convert("RGB")
        img_name = os.path.basename(img_path)

        # Lưu ảnh gốc
        output_img_path = os.path.join(output_dir, "images/train", img_name)
        output_label_path = os.path.join(output_dir, "labels/train", img_name.replace(".png", ".txt"))
        image.save(output_img_path)
        create_yolo_label(class_id, x1, y1, x2, y2, width, height, output_label_path)

    except Exception as e:
        print(f"❌ Lỗi khi xử lý ảnh {img_path}: {e}")

print("✅ Chuẩn bị dữ liệu thành công!")



✅ Chuẩn bị dữ liệu thành công!


In [ ]:
# Tạo tập validation
all_images = os.listdir(os.path.join(output_dir, "images/train"))
val_size = int(len(all_images) * 0.2)
val_images = random.sample(all_images, val_size)

for img_name in val_images:
    src_img = os.path.join(output_dir, "images/train", img_name)
    dst_img = os.path.join(output_dir, "images/val", img_name)
    shutil.move(src_img, dst_img)

    label_name = img_name.replace(".png", ".txt")
    src_label = os.path.join(output_dir, "labels/train", label_name)
    dst_label = os.path.join(output_dir, "labels/val", label_name)
    shutil.move(src_label, dst_label)

print(f"✅ Đã tạo tập validation với {val_size} ảnh!")


✅ Đã tạo tập validation với 7841 ảnh!


In [6]:
from ultralytics import YOLO

# Load mô hình YOLOv8
model = YOLO("yolov8n.pt")  

100%|██████████| 6.25M/6.25M [00:05<00:00, 1.18MB/s]


In [8]:
import torch
torch.set_num_threads(12)
print(torch.get_num_threads())  
model.train(data="../dataset/data.yaml", epochs=10, imgsz=224, batch=8, device="cpu")


12
New https://pypi.org/project/ultralytics/8.3.101 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.100  Python-3.11.0 torch-2.6.0+cpu CPU (AMD Ryzen 5 5600H with Radeon Graphics)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=../dataset/data.yaml, epochs=10, time=None, patience=100, batch=8, imgsz=224, save=True, save_period=-1, cache=False, device=cpu, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, sav

100%|██████████| 22.2M/22.2M [00:19<00:00, 1.18MB/s]


                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128

Model summary: 129 layers, 3,019,233 parameters, 3,019,217 gradients, 8.2 GFLOPs

Transferred 355/355 items from pretrained weights
TensorBoard: WARNING  TensorBoard not initialized correctly, not logging this run. runs\detect\train is not a directory
Freezing layer 'model.22.dfl.conv.weight'


train: Scanning D:\DA Thị Giác Máy Tính\Nhan Biet Bien Bao Giao Thong\dataset\labels\train.cache... 31368 images, 0 backgrounds, 0 corrupt: 100%|██████████| 31368/31368 [00:00<?, ?it/s]
val: Scanning D:\DA Thị Giác Máy Tính\Nhan Biet Bien Bao Giao Thong\dataset\labels\val.cache... 7841 images, 0 backgrounds, 0 corrupt: 100%|██████████| 7841/7841 [00:00<?, ?it/s]

...

                   all       7841       7841      0.994      0.996      0.995      0.937



10 epochs completed in 4.312 hours.
Optimizer stripped from runs\detect\train\weights\last.pt, 6.2MB
Optimizer stripped from runs\detect\train\weights\best.pt, 6.2MB

Validating runs\detect\train\weights\best.pt...
Ultralytics 8.3.100  Python-3.11.0 torch-2.6.0+cpu CPU (AMD Ryzen 5 5600H with Radeon Graphics)
Model summary (fused): 72 layers, 3,014,033 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 491/491 [02:21<00:00,  3.47it/s]


                   all       7841       7841      0.994      0.996      0.995      0.937
Gii hn tc  (20km/h)         37         37      0.993          1      0.995      0.954
Gii hn tc  (30km/h)        455        455      0.999      0.993      0.995      0.945
Gii hn tc  (50km/h)        480        480          1      0.997      0.995      0.929
Gii hn tc  (60km/h)        286        286      0.995          1      0.995      0.936
Gii hn tc  (70km/h)        362        362      0.996          1      0.995      0.937
Gii hn tc  (80km/h)        394        394      0.995       0.99      0.995      0.927
Ht gii hn tc  (80km/h)         88         88      0.997          1      0.995      0.916
Gii hn tc  (100km/h)        267        267      0.997      0.996      0.995       0.95
Gii hn tc  (120km/h)        280        280      0.996          1      0.995      0.929
              Cm vt        276        276      0.999          1      0.995      0.937
Cm xe ti trn 3.5 tn vt        420        420  

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000001EC47EAE690>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    

In [1]:
import os
import pandas as pd
import cv2
import numpy as np
import shutil
from ultralytics import YOLO
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Đường dẫn đến dữ liệu gốc
image_folder = '../data'   
label_file = '../data' 

# Đường dẫn YOLO output cho tập test
output_image_folder = '../dataset/images/test'  
output_label_folder = '../dataset/labels/test'  

# Tạo các thư mục đầu ra nếu chưa tồn tại
os.makedirs(output_image_folder, exist_ok=True)
os.makedirs(output_label_folder, exist_ok=True)

 

In [2]:
 # Đọc file CSV 
csv_path = os.path.join(label_file, "Test.csv")
df = pd.read_csv(csv_path)

# Chuyển đổi dữ liệu sang định dạng YOLO
for idx, row in df.iterrows():
    filename = row['Path']  
    width = row['Width']
    height = row['Height']
    x1 = row['Roi.X1']
    y1 = row['Roi.Y1']
    x2 = row['Roi.X2']
    y2 = row['Roi.Y2']
    class_id = row['ClassId']
    
    # Tính toán YOLO format
    x_center = ((x1 + x2) / 2) / width
    y_center = ((y1 + y2) / 2) / height
    box_width = (x2 - x1) / width
    box_height = (y2 - y1) / height
    
    # Chỉ lấy tên file (không bao gồm đường dẫn) để tạo nhãn
    base_filename = os.path.basename(filename)
    label_filename = os.path.splitext(base_filename)[0] + '.txt'
    label_path = os.path.join(output_label_folder, label_filename)
    
    # Tạo thư mục cha của label_path nếu chưa tồn tại
    os.makedirs(os.path.dirname(label_path), exist_ok=True)
    
    # Ghi file nhãn
    try:
        with open(label_path, 'w') as f:
            f.write(f"{class_id} {x_center:.6f} {y_center:.6f} {box_width:.6f} {box_height:.6f}\n")
    except Exception as e:
        print(f"Lỗi khi ghi file nhãn {label_path}: {e}")
    
    # Copy ảnh tương ứng
    src_image_path = os.path.join(image_folder, filename)
    dst_image_path = os.path.join(output_image_folder, base_filename)
    
    if os.path.exists(src_image_path):
        try:
            shutil.copy(src_image_path, dst_image_path)
        except Exception as e:
            print(f"Lỗi khi sao chép ảnh từ {src_image_path} sang {dst_image_path}: {e}")
    else:
        print(f"Không tìm thấy ảnh: {src_image_path}")

print("Hoàn tất chuyển đổi dữ liệu sang định dạng YOLO!")

Hoàn tất chuyển đổi dữ liệu sang định dạng YOLO!


In [4]:
from ultralytics import YOLO

# Tải mô hình YOLOv8 đã huấn luyện
model = YOLO('runs/detect/train/weights/best.pt')  

# Đánh giá mô hình trên tập test
metrics = model.val(data='../dataset/data.yaml', split='test')

# In kết quả đánh giá
print("🎯 Kết quả đánh giá:")
print(f"mAP@0.5: {metrics.box.map50:.4f}")          
print(f"mAP@0.5:0.95: {metrics.box.map:.4f}")      
print(f"Precision: {metrics.box.p.mean():.4f}")    
print(f"Recall: {metrics.box.r.mean():.4f}")       

Ultralytics 8.3.100  Python-3.11.0 torch-2.6.0+cpu CPU (AMD Ryzen 5 5600H with Radeon Graphics)
Model summary (fused): 72 layers, 3,014,033 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning D:\DA Thị Giác Máy Tính\Nhan Biet Bien Bao Giao Thong\dataset\labels\test... 12630 images, 0 backgrounds, 0 corrupt: 100%|██████████| 12630/12630 [00:31<00:00, 405.57it/s]


val: New cache created: D:\DA Th Gic My Tnh\Nhan Biet Bien Bao Giao Thong\dataset\labels\test.cache


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 790/790 [03:48<00:00,  3.45it/s]


                   all      12630      12630      0.958       0.94       0.97      0.901
Gii hn tc  (20km/h)         60         60      0.992          1      0.995      0.934
Gii hn tc  (30km/h)        720        720      0.999      0.996      0.995      0.936
Gii hn tc  (50km/h)        750        750      0.995      0.996      0.995      0.945
Gii hn tc  (60km/h)        450        450      0.977       0.98      0.994      0.937
Gii hn tc  (70km/h)        660        660          1      0.976      0.995      0.929
Gii hn tc  (80km/h)        630        630      0.964      0.989      0.982      0.916
Ht gii hn tc  (80km/h)        150        150      0.993      0.947      0.993      0.836
Gii hn tc  (100km/h)        450        450          1      0.999      0.995      0.941
Gii hn tc  (120km/h)        450        450      0.995      0.944      0.986      0.908
              Cm vt        480        480          1      0.989      0.995      0.937
Cm xe ti trn 3.5 tn vt        660        660  